In [33]:
#import necessary library
import pandas as pd
import requests
import csv
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup #For data scrapping

In [34]:
#For mounting the drive
#If you want to run it on your local machine then skip it and change the csv diractory as your own interest
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [35]:
#Html Request to the website.
page = requests.get("https://www.worldometers.info/coronavirus/?fbclid=IwAR0L9-HT-jDr8yXdZaeIsKT_kLkwFyGxwGGa8ezWQ2lGRrDd4Kjk_zqWtE8#news")
soup = BeautifulSoup(page.content,'html.parser')
#from the website, All we need is the table.
table = soup.findAll("table", {"id":"main_table_countries_today"})[0]
# Fatch all the rows and store in a list.
rows = table.findAll("tr")

# Store as a CSV format

with open("/content/gdrive/My Drive/Colab Notebooks/COVID-19/covid_update.csv", "wt+",encoding="utf-8", newline="") as f:
    writer = csv.writer(f)
    for row in rows:
      csv_row = []
      for cell in row.findAll(["td", "th"]):
        csv_row.append(cell.get_text())
      writer.writerow(csv_row)


# Data Exploration

In [36]:
#importing necessary library
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set() # Apply the default seaborn theme, scaling, and color palette

import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [37]:
df = pd.read_csv("covid_update.csv")

In [39]:
df.head(10)

,#,"Country,Other",TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,"Serious,Critical",Tot Cases/1M pop,Deaths/1M pop,TotalTests,Tests/\n1M pop\n,Population,Continent,1 Caseevery X ppl,1 Deathevery X ppl,1 Testevery X ppl
0,NaN,\nNorth America\n,"3,132,147","+7,829","167,687",+584,"1,370,394","+10,181","1,594,066","18,913",NaN,NaN,NaN,NaN,NaN,North America,\n,NaN,NaN
1,NaN,\nSouth America\n,"2,189,877","+3,349","83,695",+81,"1,268,518","+1,457","837,664","12,782",NaN,NaN,NaN,NaN,NaN,South America,\n,NaN,NaN
2,NaN,\nEurope\n,"2,439,014","+9,780","191,288",+262,"1,369,463","+12,518","878,263","5,720",NaN,NaN,NaN,NaN,NaN,Europe,\n,NaN,NaN
3,NaN,\nAsia\n,"2,286,528","+30,144","56,474",+744,"1,499,647","+23,451","730,407","19,345",NaN,NaN,NaN,NaN,NaN,Asia,\n,NaN,NaN
4,NaN,\nAfrica\n,"397,484","+1,170","9,930",+16,"190,617",+895,"196,937",920,NaN,NaN,NaN,NaN,NaN,Africa,\n,NaN,NaN
5,NaN,\nOceania\n,"9,476",+69,126,NaN,"8,599",NaN,751,1,NaN,NaN,NaN,NaN,NaN,Australia/Oceania,\n,NaN,NaN
6,NaN,\n\n,721,NaN,15,NaN,651,NaN,55,4,NaN,NaN,NaN,NaN,NaN,NaN,\n,NaN,NaN
7,NaN,World,"10,455,247","+52,341","509,215","+1,687","5,707,889","+48,502","4,238,143","57,685","1,341",65.3,NaN,NaN,NaN,All,\n,NaN,NaN
8,1.0,USA,"2,684,266","+2,455","128,857",+74,"1,122,655","+5,478","1,432,754","15,864","8,110",389,"33,203,107","100,312","330,996,984",North America,123,"2,569",10
9,2.0,Brazil,"1,373,006","+2,518","58,406",+21,"757,462",NaN,"557,138","8,318","6,460",275,"3,070,447","14,445","212,554,056",South America,155,"3,639",69


At first we need to name our features in a meaningful and authentic way. 

In [43]:
df.columns = ['#','Country','TotalCases','NewCases','TotalDeaths','NewDeaths','TotalRecovered','NewRecovered','ActiveCases','SeriousCritical','TotalCasesPerMillion','DeathsPerMillion','TotalTests','TestsPerMillion','Population','Continent','CaseEvery','DeathEvery','TestEvery']
df.head()

,#,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent,CaseEvery,DeathEvery,TestEvery
0,NaN,\nNorth America\n,"3,132,147","+7,829","167,687",+584,"1,370,394","+10,181","1,594,066","18,913",NaN,NaN,NaN,NaN,NaN,North America,\n,NaN,NaN
1,NaN,\nSouth America\n,"2,189,877","+3,349","83,695",+81,"1,268,518","+1,457","837,664","12,782",NaN,NaN,NaN,NaN,NaN,South America,\n,NaN,NaN
2,NaN,\nEurope\n,"2,439,014","+9,780","191,288",+262,"1,369,463","+12,518","878,263","5,720",NaN,NaN,NaN,NaN,NaN,Europe,\n,NaN,NaN
3,NaN,\nAsia\n,"2,286,528","+30,144","56,474",+744,"1,499,647","+23,451","730,407","19,345",NaN,NaN,NaN,NaN,NaN,Asia,\n,NaN,NaN
4,NaN,\nAfrica\n,"397,484","+1,170","9,930",+16,"190,617",+895,"196,937",920,NaN,NaN,NaN,NaN,NaN,Africa,\n,NaN,NaN


We've seen some columns are not that much rellivent. We'll reduce those columns

In [49]:
df = df.drop(['#','CaseEvery','DeathEvery','TestEvery'], axis=1)
df.head(10)

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
0,\nNorth America\n,"3,132,147","+7,829","167,687",+584,"1,370,394","+10,181","1,594,066","18,913",NaN,NaN,NaN,NaN,NaN,North America
1,\nSouth America\n,"2,189,877","+3,349","83,695",+81,"1,268,518","+1,457","837,664","12,782",NaN,NaN,NaN,NaN,NaN,South America
2,\nEurope\n,"2,439,014","+9,780","191,288",+262,"1,369,463","+12,518","878,263","5,720",NaN,NaN,NaN,NaN,NaN,Europe
3,\nAsia\n,"2,286,528","+30,144","56,474",+744,"1,499,647","+23,451","730,407","19,345",NaN,NaN,NaN,NaN,NaN,Asia
4,\nAfrica\n,"397,484","+1,170","9,930",+16,"190,617",+895,"196,937",920,NaN,NaN,NaN,NaN,NaN,Africa
5,\nOceania\n,"9,476",+69,126,NaN,"8,599",NaN,751,1,NaN,NaN,NaN,NaN,NaN,Australia/Oceania
6,\n\n,721,NaN,15,NaN,651,NaN,55,4,NaN,NaN,NaN,NaN,NaN,NaN
7,World,"10,455,247","+52,341","509,215","+1,687","5,707,889","+48,502","4,238,143","57,685","1,341",65.3,NaN,NaN,NaN,All
8,USA,"2,684,266","+2,455","128,857",+74,"1,122,655","+5,478","1,432,754","15,864","8,110",389,"33,203,107","100,312","330,996,984",North America
9,Brazil,"1,373,006","+2,518","58,406",+21,"757,462",NaN,"557,138","8,318","6,460",275,"3,070,447","14,445","212,554,056",South America


The First 8 Rows data is continantal data. We will deal with only countries data. So We're going to delete first 8 row from this dataset. 

In [50]:
df = df.iloc[8:]
df.head()


,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
8,USA,"2,684,266","+2,455","128,857",+74,"1,122,655","+5,478","1,432,754","15,864","8,110",389,"33,203,107","100,312","330,996,984",North America
9,Brazil,"1,373,006","+2,518","58,406",+21,"757,462",NaN,"557,138","8,318","6,460",275,"3,070,447","14,445","212,554,056",South America
10,Russia,"647,849","+6,693","9,320",+154,"412,650","+9,220","225,879","2,300","4,439",64,"19,562,440","134,050","145,934,449",Europe
11,India,"573,598","+6,062","17,008",+104,"339,125","+3,854","217,465","8,944",416,12,"8,608,654","6,238","1,379,937,465",Asia
12,UK,"311,965",NaN,"43,575",NaN,NaN,NaN,NaN,276,"4,595",642,"9,290,215","136,852","67,885,079",Europe


We can see the index start from 8. So we're going to reset the index number.

In [51]:
df = df.reset_index(drop=True)
df.head()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
0,USA,"2,684,266","+2,455","128,857",+74,"1,122,655","+5,478","1,432,754","15,864","8,110",389,"33,203,107","100,312","330,996,984",North America
1,Brazil,"1,373,006","+2,518","58,406",+21,"757,462",NaN,"557,138","8,318","6,460",275,"3,070,447","14,445","212,554,056",South America
2,Russia,"647,849","+6,693","9,320",+154,"412,650","+9,220","225,879","2,300","4,439",64,"19,562,440","134,050","145,934,449",Europe
3,India,"573,598","+6,062","17,008",+104,"339,125","+3,854","217,465","8,944",416,12,"8,608,654","6,238","1,379,937,465",Asia
4,UK,"311,965",NaN,"43,575",NaN,NaN,NaN,NaN,276,"4,595",642,"9,290,215","136,852","67,885,079",Europe


In [52]:
#Data discription
df.describe()

,Country,TotalCases,NewCases,TotalDeaths,NewDeaths,TotalRecovered,NewRecovered,ActiveCases,SeriousCritical,TotalCasesPerMillion,DeathsPerMillion,TotalTests,TestsPerMillion,Population,Continent
count,223,223,94,223,59,218,79,219,135,214,184,192,192,215,220
unique,216,209,73,143,33,207,71,179,74,202,101,192,192,214,7
top,Total:,77,+2,,+1,19,+3,0,1,19,3,"6,273","105,911",,Africa
freq,8,3,8,30,10,3,5,25,24,3,10,1,1,2,58


From this Description We got some interesting instinct. <br>
### 1: Tatal affected countries = 215 
### 2: Most affectected Continent = Africa
### 3: Total affect countries in Africa = 58

In [54]:
df.describe

<bound method NDFrame.describe of     Country  TotalCases  ...      Population          Continent
0       USA   2,684,266  ...    330,996,984       North America
1    Brazil   1,373,006  ...    212,554,056       South America
2    Russia     647,849  ...    145,934,449              Europe
3     India     573,598  ...  1,379,937,465                Asia
4        UK     311,965  ...     67,885,079              Europe
..      ...         ...  ...             ...                ...
218  Total:   2,286,528  ...             NaN               Asia
219  Total:     397,484  ...             NaN             Africa
220  Total:       9,476  ...             NaN  Australia/Oceania
221  Total:         721  ...             NaN                NaN
222  Total:  10,455,247  ...             NaN                All

[223 rows x 15 columns]>